# Import Libraries

In [1]:
import sys
import os
import sqlite3
import warnings
import json

import pandas as pd
import numpy as np

import optuna
import shap
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier # Changed from ExtraTreesClassifier
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, classification_report, confusion_matrix
from sklearn.feature_selection import RFECV


warnings.filterwarnings('ignore')
np.random.seed(42)

/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Database

In [2]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

DATABASE_PATH = "./database/database.db"

RESULTS_DIR = 'results'

os.makedirs(RESULTS_DIR, exist_ok=True)

conn = sqlite3.connect(DATABASE_PATH)

data = pd.read_sql(sql='SELECT * FROM data', con=conn)

conn.close()

In [3]:
def run(di, di_scale, selected_stations, stations_group_name, start_date, end_date):
    
    group_dir = os.path.join(RESULTS_DIR, stations_group_name)
    os.makedirs(group_dir, exist_ok=True)
    
    scale_dir = os.path.join(group_dir, f'{di}_{di_scale}')
    os.makedirs(scale_dir, exist_ok=True)
    
    # Select Columns
    selected_columns = [
        'Station_Name', 'Station_ID',
        'Station_Latitude', 'Station_Longitude', 'Station_Elevation',
        'Date',
        f'{di}_{di_scale}',
        f'GPM_{di}_{di_scale}',
        f'ERA5_{di}_{di_scale}',
        'ERA5_Precipitation',
        'GPM_Precipitation',
        'PET_MOD16A2GF',
        'NDVI', 'EVI',
        'LSTDay', 'LSTNight', 'LST',
        'PCI_GPM', 'PCI_ERA5',
        'VCI', 'TCI', 'VHI',
        'CI_GPM', 'CI_ERA5',
    ]
    
    df = data\
    .filter(items=selected_columns)\
        .query("Station_Name in @selected_stations and Date >= @start_date and Date < @end_date")

    # Date, Year, Month
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    # SPI and SPEI Class
    df[f'{di}_{di_scale}_Class'] = pd.cut(df[f'{di}_{di_scale}'], bins=[-10, -2, -1.5, -1, 1, 1.5, 2, 10], labels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW'])

    # Month Sin & Cos
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
            
    # LST Diff
    df['LST_Diff'] = df['LSTDay'] - df['LSTNight']

    # Convert to Category
    df['Station_ID'] = df['Station_ID'].astype('category')
    df['Year'] = df['Year'].astype('category')
    df['Month'] = df['Month'].astype('category')
    df[f'{di}_{di_scale}_Class'] = df[f'{di}_{di_scale}_Class'].astype('category')
    
    df.dropna(inplace=True)
    df.sort_values(by=['Station_ID', 'Year', 'Month'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    selected_columns_lag_roll = [
        'ERA5_Precipitation',
        'GPM_Precipitation',
        f'GPM_{di}_{di_scale}',
        f'ERA5_{di}_{di_scale}',
        'PET_MOD16A2GF',
        'NDVI', 'EVI',
        'LSTDay', 'LSTNight', 'LST',
        'PCI_GPM', 'PCI_ERA5',
        'VCI', 'TCI', 'VHI',
        'CI_GPM', 'CI_ERA5',
    ]

    # Add Lag
    for lag in range(1, 4):
        for col in selected_columns_lag_roll:
            df[f'{col}_lag_{lag}'] = df.groupby('Station_ID', observed=False)[col].shift(lag)


    # # Add Mean and Std Roll
    # for r in [3, 6]:
    #     for col in selected_columns_lag_roll:
    #         df[f'{col}_roll_mean_{r}'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=r, min_periods=1).mean())
    #         df[f'{col}_roll_std_{r}'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=r, min_periods=1).std())
    
    
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # Define columns to exclude from features
    EXCLUDE_COLS = ['Station_Name', 'Station_ID', 'Date', f'{di}_{di_scale}_Class', f'{di}_{di_scale}', 'Year', 'Month']
    FEATURES = [col for col in df.columns if col not in EXCLUDE_COLS]

    # Station-wise standardization
    df_scaled = df.copy()
    for station in df_scaled['Station_Name'].unique():
        station_mask = df_scaled['Station_Name'] == station
        scaler = StandardScaler()
        df_scaled.loc[station_mask, FEATURES] = scaler.fit_transform(df_scaled.loc[station_mask, FEATURES])


    class_counts = df_scaled[f'{di}_{di_scale}_Class'].value_counts()
    classes_to_remove = class_counts[class_counts < 2].index

    if not classes_to_remove.empty:
        print(f"Removing classes with fewer than 2 samples: {classes_to_remove.tolist()}")
        
        # Filter the dataframe to exclude these rare classes
        df_filtered = df_scaled[~df_scaled[f'{di}_{di_scale}_Class'].isin(classes_to_remove)].copy()
        
        # Redefine X and y from the filtered dataframe
        X = df_filtered[FEATURES]
        y = df_filtered[f'{di}_{di_scale}_Class']
        
        print(f"Data shape after removing rare classes: {X.shape}")
        print("\n--- Final Class Distribution ---")
        print(y.value_counts())
    else:
        print("No classes with fewer than 2 samples found.")
        X = df_scaled[FEATURES]
        y = df_scaled[f'{di}_{di_scale}_Class']

    # Encode the target variable
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y_encoded,
        test_size=0.30,
        random_state=42,
        stratify=y_encoded
    )

    print(f"Target Classes: {le.classes_}")
    print(f"Training set shape: {X_train.shape}")
    print(f"Testing set shape: {X_test.shape}")


    # The estimator that will be used by RFECV
    estimator = RandomForestClassifier(
        random_state=42,
        n_jobs=-1
    )

    # The RFECV object
    rfecv = RFECV(
        estimator=estimator,
        step=1,
        cv=StratifiedKFold(5),
        scoring='f1_macro',
        n_jobs=-1
    )

    # Fit RFECV on the training data
    rfecv.fit(X_train, y_train)

    print(f"Optimal number of features found: {rfecv.n_features_}")

    # Get the final selected features
    final_features = X_train.columns[rfecv.support_]
    print(f"Selected features: {final_features.tolist()}")
    
    features_path = os.path.join(scale_dir, 'selected_features.txt')
    with open(features_path, 'w') as f:
        for feature in final_features:
            f.write(f"{feature}\n")
    print(f"Selected features saved to {features_path}")

    # --- Plot Feature Importances ---
    # The `rfecv.estimator_` attribute is the model trained on the full set of features
    # during the last step of the cross-validation process. We can use its feature importances.
    importances = rfecv.estimator_.feature_importances_
    feature_importance_df = pd.DataFrame({
        'Feature': final_features,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)

    # Plot only the selected features
    selected_feature_importances = feature_importance_df[feature_importance_df['Feature'].isin(final_features)]

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=selected_feature_importances)
    plt.title('Feature Importances for Selected Features (from RFECV)')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    fig_path = os.path.join(scale_dir, 'feature_importances.png')
    plt.savefig(fig_path)
    plt.close()


    # Update training and testing sets with selected features
    X_train_final = X_train[final_features]
    X_test_final = X_test[final_features]
    
    
    def objective(trial):
        """Define the objective function for Optuna to optimize."""
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 5, 50),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'max_features': trial.suggest_float('max_features', 0.1, 1.0),
            'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
            'random_state': 42,
            'n_jobs': -1
        }
        
        model = RandomForestClassifier(**params)
        
        # Stratified K-Fold Cross-Validation
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        f1_scores = []
        
        for train_idx, val_idx in skf.split(X_train_final, y_train):
            X_train_fold, X_val_fold = X_train_final.iloc[train_idx], X_train_final.iloc[val_idx]
            y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
            
            model.fit(X_train_fold, y_train_fold)
            preds = model.predict(X_val_fold)
            f1_scores.append(f1_score(y_val_fold, preds, average='macro'))
            
        return np.mean(f1_scores)

    # Create a study object and optimize
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50, timeout=600) # 50 trials or 10 minutes

    # Save best parameters
    best_params = study.best_params 
    params_path = os.path.join(scale_dir, 'best_params.json')
    with open(params_path, 'w') as f:
        json.dump(best_params, f, indent=4)
    print(f"Best parameters saved to {params_path}")

    print(f"Best trial found: {study.best_trial.number}")
    print(f"Best Macro-F1 Score: {study.best_value}")
    print("Best Hyperparameters:")
    print(study.best_params)



    # Get best hyperparameters
    best_params = study.best_params
    best_params['random_state'] = 42
    best_params['n_jobs'] = -1

    # Train the final model
    final_model = RandomForestClassifier(**best_params)
    final_model.fit(X_train_final, y_train)

    # Make predictions on the test set
    y_pred = final_model.predict(X_test_final)

    # Save evaluation metrics
    report = classification_report(y_test, y_pred, target_names=le.classes_)
    accuracy = accuracy_score(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    kappa = cohen_kappa_score(y_test, y_pred)
    
    metrics_path = os.path.join(scale_dir, 'evaluation_metrics.txt')
    with open(metrics_path, 'w') as f:
        f.write(f"Overall Accuracy: {accuracy:.4f}\n")
        f.write(f"Macro-F1 Score: {macro_f1:.4f}\n")
        f.write(f"Cohen's Kappa: {kappa:.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(report)
    print(f"Evaluation metrics saved to {metrics_path}")

    print(f"Overall Accuracy: {accuracy:.4f}")
    print(f"Macro-F1 Score: {macro_f1:.4f}")
    print(f"Cohen's Kappa: {kappa:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))


    # --- Plot Confusion Matrix ---
    print("\n--- Plotting Confusion Matrix ---")
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    fig_path = os.path.join(scale_dir, 'confusion_matrix.png')
    plt.savefig(fig_path)
    plt.close()
        

    # # Create SHAP explainer for the final model
    # # Using a subset of the training data for the background is a common practice for performance
    # explainer = shap.TreeExplainer(final_model, X_train_final.sample(100, random_state=42))
    # shap_values = explainer.shap_values(X_test_final)

    # try:
    #     # SHAP Summary Plot (Bar)
    #     print("\nGenerating SHAP Summary Bar Plot...")
    #     plt.figure()
    #     shap.summary_plot(shap_values, X_test_final, plot_type="bar", class_names=le.classes_, show=False)
    #     plt.title("SHAP Summary Plot (Global Feature Importance)")
    #     plt.tight_layout()
    #     fig_path = os.path.join(scale_dir, 'shap_summary_bar.png')
    #     plt.savefig(fig_path, bbox_inches='tight')
    #     plt.close()
    # except:
    #     print("An exception occurred: Generating SHAP Summary Bar Plot...")
        
    # try:
    #     # SHAP Summary Plot (Beeswarm)
    #     print("\nGenerating SHAP Summary Beeswarm Plot...")
    #     plt.figure()
    #     shap.summary_plot(shap_values, X_test_final, class_names=le.classes_, show=False,  plot_type="violin")
    #     plt.title("SHAP Summary Plot (Beeswarm)")
    #     plt.tight_layout()
    #     fig_path = os.path.join(scale_dir, 'shap_summary_beeswarm.png')
    #     plt.savefig(fig_path, bbox_inches='tight')
    #     plt.close()
    # except:
    #     print("An exception occurred: SHAP Summary Plot (Beeswarm)")
    
    return None



for di in ['SPI', 'SPEI']:
    for di_scale in [1, 3, 6, 9, 12, 18, 24]:
        for stations_group_name in ['ALL', 'C1', 'C2', 'C3']: 

            if stations_group_name == 'ALL':
                selected_stations = ['Ramsar', 'Nowshahr', 'Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht', 'Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']
            elif stations_group_name == 'C3':
                selected_stations = ['Ramsar', 'Nowshahr']
            elif stations_group_name == 'C2':
                selected_stations = ['Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht']
            elif stations_group_name == 'C1':
                selected_stations = ['Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']
            else:
                selected_stations = ['Ramsar', 'Nowshahr', 'Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht', 'Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']

            ex_dir = os.path.join(RESULTS_DIR, stations_group_name, f'{di}_{di_scale}')
            if os.path.exists(ex_dir):
                continue 
            
            run(
                di=di,
                di_scale=di_scale,
                selected_stations=selected_stations,
                stations_group_name=stations_group_name,
                start_date='2006-09', 
                end_date='2023-10'
            )

No classes with fewer than 2 samples found.
Target Classes: ['ED' 'EW' 'MD' 'MW' 'NN' 'SD' 'VW']
Training set shape: (2121, 74)
Testing set shape: (909, 74)
Optimal number of features found: 40
Selected features: ['GPM_SPI_1', 'ERA5_SPI_1', 'ERA5_Precipitation', 'GPM_Precipitation', 'PET_MOD16A2GF', 'NDVI', 'LSTDay', 'LST', 'PCI_GPM', 'PCI_ERA5', 'TCI', 'VHI', 'CI_GPM', 'CI_ERA5', 'LST_Diff', 'ERA5_Precipitation_lag_1', 'GPM_SPI_1_lag_1', 'PET_MOD16A2GF_lag_1', 'NDVI_lag_1', 'EVI_lag_1', 'PCI_GPM_lag_1', 'PCI_ERA5_lag_1', 'TCI_lag_1', 'VHI_lag_1', 'CI_GPM_lag_1', 'ERA5_Precipitation_lag_2', 'GPM_SPI_1_lag_2', 'ERA5_SPI_1_lag_2', 'EVI_lag_2', 'LSTDay_lag_2', 'LSTNight_lag_2', 'VCI_lag_2', 'TCI_lag_2', 'ERA5_Precipitation_lag_3', 'GPM_Precipitation_lag_3', 'EVI_lag_3', 'LSTNight_lag_3', 'PCI_GPM_lag_3', 'TCI_lag_3', 'CI_GPM_lag_3']
Selected features saved to results/ALL/SPI_1/selected_features.txt


[I 2025-08-24 02:18:03,047] A new study created in memory with name: no-name-bd9535d7-0636-462d-92df-201bfb1adb4c
[I 2025-08-24 02:18:04,449] Trial 0 finished with value: 0.24628435914557043 and parameters: {'n_estimators': 82, 'max_depth': 47, 'min_samples_split': 14, 'min_samples_leaf': 11, 'max_features': 0.2727719747457127, 'criterion': 'entropy'}. Best is trial 0 with value: 0.24628435914557043.
[I 2025-08-24 02:18:11,635] Trial 1 finished with value: 0.3078228804250142 and parameters: {'n_estimators': 255, 'max_depth': 48, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 0.682619468522241, 'criterion': 'entropy'}. Best is trial 1 with value: 0.3078228804250142.
[I 2025-08-24 02:18:15,422] Trial 2 finished with value: 0.23928876792847217 and parameters: {'n_estimators': 64, 'max_depth': 19, 'min_samples_split': 6, 'min_samples_leaf': 13, 'max_features': 0.25091103216237043, 'criterion': 'gini'}. Best is trial 1 with value: 0.3078228804250142.
[I 2025-08-24 02:18:18,3

Best parameters saved to results/ALL/SPI_1/best_params.json
Best trial found: 43
Best Macro-F1 Score: 0.3296598051553986
Best Hyperparameters:
{'n_estimators': 223, 'max_depth': 33, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 0.9119958078956494, 'criterion': 'gini'}
Evaluation metrics saved to results/ALL/SPI_1/evaluation_metrics.txt
Overall Accuracy: 0.6777
Macro-F1 Score: 0.2601
Cohen's Kappa: 0.2374

Classification Report:
              precision    recall  f1-score   support

          ED       0.40      0.15      0.21        41
          EW       0.23      0.14      0.18        21
          MD       0.14      0.05      0.08        55
          MW       0.36      0.21      0.26        97
          NN       0.74      0.94      0.83       608
          SD       0.00      0.00      0.00        27
          VW       0.36      0.20      0.26        60

    accuracy                           0.68       909
   macro avg       0.32      0.24      0.26       909
weighted 

[I 2025-08-24 02:28:29,491] A new study created in memory with name: no-name-0572038e-8b2a-40f0-8306-86c1205a3853
[I 2025-08-24 02:28:31,135] Trial 0 finished with value: 0.26839229072956095 and parameters: {'n_estimators': 119, 'max_depth': 37, 'min_samples_split': 3, 'min_samples_leaf': 17, 'max_features': 0.5328639653483696, 'criterion': 'gini'}. Best is trial 0 with value: 0.26839229072956095.
[I 2025-08-24 02:28:34,462] Trial 1 finished with value: 0.2647398112750366 and parameters: {'n_estimators': 233, 'max_depth': 48, 'min_samples_split': 12, 'min_samples_leaf': 20, 'max_features': 0.887945590179658, 'criterion': 'entropy'}. Best is trial 0 with value: 0.26839229072956095.
[I 2025-08-24 02:28:36,279] Trial 2 finished with value: 0.32489478160369034 and parameters: {'n_estimators': 140, 'max_depth': 48, 'min_samples_split': 12, 'min_samples_leaf': 5, 'max_features': 0.3095369409903025, 'criterion': 'gini'}. Best is trial 2 with value: 0.32489478160369034.
[I 2025-08-24 02:28:40,

Best parameters saved to results/C1/SPI_1/best_params.json
Best trial found: 33
Best Macro-F1 Score: 0.3435156243131224
Best Hyperparameters:
{'n_estimators': 85, 'max_depth': 48, 'min_samples_split': 14, 'min_samples_leaf': 2, 'max_features': 0.5442174152364422, 'criterion': 'gini'}
Evaluation metrics saved to results/C1/SPI_1/evaluation_metrics.txt
Overall Accuracy: 0.6722
Macro-F1 Score: 0.3041
Cohen's Kappa: 0.2884

Classification Report:
              precision    recall  f1-score   support

          ED       0.44      0.19      0.27        21
          EW       0.40      0.17      0.24        12
          MD       0.07      0.03      0.05        30
          MW       0.44      0.40      0.42        50
          NN       0.76      0.90      0.82       322
          SD       0.00      0.00      0.00        16
          VW       0.40      0.29      0.34        34

    accuracy                           0.67       485
   macro avg       0.36      0.28      0.30       485
weighted av

...

Best parameters saved to results/C2/SPEI_24/best_params.json
Best trial found: 23
Best Macro-F1 Score: 0.44050167786657424
Best Hyperparameters:
{'n_estimators': 170, 'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': 0.701420722938032, 'criterion': 'gini'}
Evaluation metrics saved to results/C2/SPEI_24/evaluation_metrics.txt
Overall Accuracy: 0.7881
Macro-F1 Score: 0.4887
Cohen's Kappa: 0.5562

Classification Report:
              precision    recall  f1-score   support

          ED       1.00      0.33      0.50         3
          EW       0.00      0.00      0.00         1
          MD       0.43      0.17      0.24        18
          MW       0.68      0.51      0.58        37
          NN       0.84      0.95      0.89       179
          SD       0.50      0.54      0.52        13
          VW       0.71      0.67      0.69        18

    accuracy                           0.79       269
   macro avg       0.59      0.45      0.49       269
weighte

[I 2025-08-24 06:16:59,644] A new study created in memory with name: no-name-c5385b52-0085-441d-8d6a-627b9b67804a


Optimal number of features found: 8
Selected features: ['GPM_SPEI_24', 'ERA5_SPEI_24', 'LST_Diff', 'GPM_SPEI_24_lag_1', 'ERA5_SPEI_24_lag_1', 'ERA5_SPEI_24_lag_2', 'NDVI_lag_2', 'ERA5_SPEI_24_lag_3']
Selected features saved to results/C3/SPEI_24/selected_features.txt


[I 2025-08-24 06:17:06,227] Trial 0 finished with value: 0.21947253053433968 and parameters: {'n_estimators': 463, 'max_depth': 38, 'min_samples_split': 14, 'min_samples_leaf': 5, 'max_features': 0.8427159986343469, 'criterion': 'gini'}. Best is trial 0 with value: 0.21947253053433968.
[I 2025-08-24 06:17:09,173] Trial 1 finished with value: 0.21458721687151167 and parameters: {'n_estimators': 382, 'max_depth': 37, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_features': 0.7808175951597492, 'criterion': 'gini'}. Best is trial 0 with value: 0.21947253053433968.
[I 2025-08-24 06:17:12,563] Trial 2 finished with value: 0.211222321538111 and parameters: {'n_estimators': 442, 'max_depth': 34, 'min_samples_split': 19, 'min_samples_leaf': 14, 'max_features': 0.20645313941956195, 'criterion': 'gini'}. Best is trial 0 with value: 0.21947253053433968.
[I 2025-08-24 06:17:14,190] Trial 3 finished with value: 0.21135305885305886 and parameters: {'n_estimators': 194, 'max_depth': 30, 'min_s

Best parameters saved to results/C3/SPEI_24/best_params.json
Best trial found: 41
Best Macro-F1 Score: 0.34249134364806005
Best Hyperparameters:
{'n_estimators': 198, 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 0.7210416616373629, 'criterion': 'entropy'}
Evaluation metrics saved to results/C3/SPEI_24/evaluation_metrics.txt
Overall Accuracy: 0.5926
Macro-F1 Score: 0.3401
Cohen's Kappa: 0.2533

Classification Report:
              precision    recall  f1-score   support

          ED       0.50      1.00      0.67         2
          MD       0.00      0.00      0.00         8
          MW       0.34      0.59      0.43        17
          NN       0.74      0.74      0.74        69
          SD       0.25      0.17      0.20         6
          VW       0.00      0.00      0.00         6

    accuracy                           0.59       108
   macro avg       0.31      0.42      0.34       108
weighted avg       0.55      0.59      0.56       108


-

# Select Columns and Rows

In [ ]:
# Select Drought Index and Period
di = 'SPEI'
di_scale = 3

# Select Stations
selected_stations = ['Ramsar', 'Nowshahr', 'Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht', 'Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']
# selected_stations = ['Ramsar', 'Nowshahr']
# selected_stations = ['Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht']
# selected_stations = ['Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']

# Select Columns
selected_columns = [
    'Station_Name', 'Station_ID',
    'Station_Latitude', 'Station_Longitude', 'Station_Elevation',
    'Date',
    f'{di}_{di_scale}',
    f'GPM_{di}_{di_scale}',
    f'ERA5_{di}_{di_scale}',
    'ERA5_Precipitation',
    'GPM_Precipitation',
    'PET_MOD16A2GF',
    'NDVI', 'EVI',
    'LSTDay', 'LSTNight', 'LST',
    'PCI_GPM', 'PCI_ERA5',
    'VCI', 'TCI', 'VHI',
    'CI_GPM', 'CI_ERA5',
]

# Select Start and End Date
start_date = '2006-09'
end_date = '2023-10'

# Filter Data

In [ ]:
df = data\
    .filter(items=selected_columns)\
        .query("Station_Name in @selected_stations and Date >= @start_date and Date < @end_date")

# Add Some Columns

In [ ]:
# Date, Year, Month
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# SPI and SPEI Class
df[f'{di}_{di_scale}_Class'] = pd.cut(df[f'{di}_{di_scale}'], bins=[-10, -2, -1.5, -1, 1, 1.5, 2, 10], labels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW'])

# Month Sin & Cos
df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
        
# LST Diff
df['LST_Diff'] = df['LSTDay'] - df['LSTNight']

# Convert to Category
df['Station_ID'] = df['Station_ID'].astype('category')
df['Year'] = df['Year'].astype('category')
df['Month'] = df['Month'].astype('category')
df[f'{di}_{di_scale}_Class'] = df[f'{di}_{di_scale}_Class'].astype('category')

In [ ]:
df.dropna(inplace=True)
df.sort_values(by=['Station_ID', 'Year', 'Month'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
selected_columns_lag_roll = [
    'ERA5_Precipitation',
    'GPM_Precipitation',
    f'GPM_{di}_{di_scale}',
    f'ERA5_{di}_{di_scale}',
    'PET_MOD16A2GF',
    'NDVI', 'EVI',
    'LSTDay', 'LSTNight', 'LST',
    'PCI_GPM', 'PCI_ERA5',
    'VCI', 'TCI', 'VHI',
    'CI_GPM', 'CI_ERA5',
]

# Add Lag
for lag in range(1, 4):
    for col in selected_columns_lag_roll:
        df[f'{col}_lag_{lag}'] = df.groupby('Station_ID', observed=False)[col].shift(lag)


# Add Mean and Std Roll
for r in [3, 6]:
    for col in selected_columns_lag_roll:
        df[f'{col}_roll_mean_{r}'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=r, min_periods=1).mean())
        df[f'{col}_roll_std_{r}'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=r, min_periods=1).std())

# Remove all NA

In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Station-wise Standardization

In [ ]:
# Define columns to exclude from features
EXCLUDE_COLS = ['Station_Name', 'Station_ID', 'Date', f'{di}_{di_scale}_Class', f'{di}_{di_scale}', 'Year', 'Month']
FEATURES = [col for col in df.columns if col not in EXCLUDE_COLS]

# Station-wise standardization
df_scaled = df.copy()
for station in df_scaled['Station_Name'].unique():
    station_mask = df_scaled['Station_Name'] == station
    scaler = StandardScaler()
    df_scaled.loc[station_mask, FEATURES] = scaler.fit_transform(df_scaled.loc[station_mask, FEATURES])


class_counts = df_scaled[f'{di}_{di_scale}_Class'].value_counts()
classes_to_remove = class_counts[class_counts < 2].index

if not classes_to_remove.empty:
    print(f"Removing classes with fewer than 2 samples: {classes_to_remove.tolist()}")
    
    # Filter the dataframe to exclude these rare classes
    df_filtered = df_scaled[~df_scaled[f'{di}_{di_scale}_Class'].isin(classes_to_remove)].copy()
    
    # Redefine X and y from the filtered dataframe
    X = df_filtered[FEATURES]
    y = df_filtered[f'{di}_{di_scale}_Class']
    
    print(f"Data shape after removing rare classes: {X.shape}")
    print("\n--- Final Class Distribution ---")
    print(y.value_counts())
else:
    print("No classes with fewer than 2 samples found.")
    X = df_scaled[FEATURES]
    y = df_scaled[f'{di}_{di_scale}_Class']

# Encode the target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.30,
    random_state=42,
    stratify=y_encoded
)

print(f"Target Classes: {le.classes_}")
print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

# Feature Selection with RFECV

In [ ]:
# The estimator that will be used by RFECV
estimator = RandomForestClassifier(
    random_state=42,
    n_jobs=-1
)

# The RFECV object
rfecv = RFECV(
    estimator=estimator,
    step=1,
    cv=StratifiedKFold(5),
    scoring='f1_macro',
    n_jobs=-1
)

# Fit RFECV on the training data
rfecv.fit(X_train, y_train)

print(f"Optimal number of features found: {rfecv.n_features_}")

# Get the final selected features
final_features = X_train.columns[rfecv.support_]
print(f"Selected features: {final_features.tolist()}")

# --- Plot Feature Importances ---
# The `rfecv.estimator_` attribute is the model trained on the full set of features
# during the last step of the cross-validation process. We can use its feature importances.
importances = rfecv.estimator_.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': final_features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Plot only the selected features
selected_feature_importances = feature_importance_df[feature_importance_df['Feature'].isin(final_features)]

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=selected_feature_importances)
plt.title('Feature Importances for Selected Features (from RFECV)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


# Update training and testing sets with selected features
X_train_final = X_train[final_features]
X_test_final = X_test[final_features]

# Hyperparameter Tuning with Optuna

In [ ]:
def objective(trial):
    """Define the objective function for Optuna to optimize."""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'random_state': 42,
        'n_jobs': -1
    }
    
    model = RandomForestClassifier(**params)
    
    # Stratified K-Fold Cross-Validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = []
    
    for train_idx, val_idx in skf.split(X_train_final, y_train):
        X_train_fold, X_val_fold = X_train_final.iloc[train_idx], X_train_final.iloc[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        model.fit(X_train_fold, y_train_fold)
        preds = model.predict(X_val_fold)
        f1_scores.append(f1_score(y_val_fold, preds, average='macro'))
        
    return np.mean(f1_scores)

# Create a study object and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, timeout=600) # 50 trials or 10 minutes

print(f"Best trial found: {study.best_trial.number}")
print(f"Best Macro-F1 Score: {study.best_value}")
print("Best Hyperparameters:")
print(study.best_params)


# Model Training & Evaluation

In [ ]:
# Get best hyperparameters
best_params = study.best_params
best_params['random_state'] = 42
best_params['n_jobs'] = -1

# Train the final model
final_model = RandomForestClassifier(**best_params)
final_model.fit(X_train_final, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test_final)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')
kappa = cohen_kappa_score(y_test, y_pred)

print(f"Overall Accuracy: {accuracy:.4f}")
print(f"Macro-F1 Score: {macro_f1:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


# --- Plot Confusion Matrix ---
print("\n--- Plotting Confusion Matrix ---")
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Explainability with SHAP

In [ ]:
# Create SHAP explainer for the final model
# Using a subset of the training data for the background is a common practice for performance
explainer = shap.TreeExplainer(final_model, X_train_final.sample(100, random_state=42))
shap_values = explainer.shap_values(X_test_final)

# SHAP Summary Plot (Bar)
print("\nGenerating SHAP Summary Bar Plot...")
plt.figure()
shap.summary_plot(shap_values, X_test_final, plot_type="bar", class_names=le.classes_, show=False)
plt.title("SHAP Summary Plot (Global Feature Importance)")
plt.tight_layout()
plt.show()

# SHAP Summary Plot (Beeswarm)
print("\nGenerating SHAP Summary Beeswarm Plot...")
plt.figure()
shap.summary_plot(shap_values, X_test_final, class_names=le.classes_, show=False,  plot_type="violin")
plt.title("SHAP Summary Plot (Beeswarm)")
plt.tight_layout()
plt.show()